In [18]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import gradient_descent_v2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


NUM_CLASSES = 11
TRAIN_PATH = 'dataset/train/'
TEST_PATH = 'dataset/test/'

FC_NUMS = 4096

FREEZE_LAYERS = 17

IMAGE_SIZE = 224

base_model = VGG19(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model.compile(optimizer=gradient_descent_v2.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator(
	zoom_range = 0.7,
    rescale = 1. /255.,
    horizontal_flip = True,
    vertical_flip= True,
    width_shift_range = 0.2,
    height_shift_range=0.2,
    validation_split=0.2,
	rotation_range=90
)
train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='categorical')
filepath = 'model_sample/sewer_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

history_ft = model.fit_generator(train_generator, epochs=50, callbacks=[checkpoint])
model.save('model_sample/sewer_weight_final.h5')

Found 550 images belonging to 11 classes.
Epoch 1/50


2022-05-23 01:50:10.674991: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 12s 576ms/step - loss: 2.4252 - accuracy: 0.0932

Epoch 00001: loss improved from inf to 2.42818, saving model to model_sample/sewer_weight.h5
Epoch 2/50
18/18 [==============================] - 11s 590ms/step - loss: 2.3825 - accuracy: 0.1021

Epoch 00002: loss improved from 2.42818 to 2.37171, saving model to model_sample/sewer_weight.h5
Epoch 3/50
18/18 [==============================] - 11s 587ms/step - loss: 2.3048 - accuracy: 0.2086

Epoch 00003: loss improved from 2.37171 to 2.28504, saving model to model_sample/sewer_weight.h5
Epoch 4/50
18/18 [==============================] - 11s 584ms/step - loss: 2.1728 - accuracy: 0.2720

Epoch 00004: loss improved from 2.28504 to 2.18153, saving model to model_sample/sewer_weight.h5
Epoch 5/50
18/18 [==============================] - 11s 574ms/step - loss: 2.0459 - accuracy: 0.3097

Epoch 00005: loss improved from 2.18153 to 2.02459, saving model to model_sample/sewer_weight.h5
Epoch 6/50
18/18 [==

In [25]:

from keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.initializers import glorot_uniform
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

print("-- Evaluate --")
valid_dir = 'dataset/sd/'
model_path = 'model/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

IMAGE_SIZE = 224

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


valid_datagen = ImageDataGenerator(rescale = 1./255)


# 'valid_dir' 디렉토리에 저장된 이미지 전처리
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='categorical')

scores = model.evaluate_generator(valid_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
Found 75 images belonging to 11 classes.


2022-05-23 02:20:00.926769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


accuracy: 22.67%
